In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kriti-final/train.csv
/kaggle/input/kriti-final/test.csv
/kaggle/input/bert-base-uncased/config.json
/kaggle/input/bert-base-uncased/tokenizer.json
/kaggle/input/bert-base-uncased/tokenizer_config.json
/kaggle/input/bert-base-uncased/model.safetensors
/kaggle/input/bert-base-uncased/special_tokens_map.json
/kaggle/input/bert-base-uncased/vocab.txt
/kaggle/input/sample-sub/sample_submission.csv


In [2]:
!pip install transformers

In [3]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys

In [5]:
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 2 GPU(s) available.
We will use the GPU: Tesla T4


In [6]:
train_path = '/kaggle/input/kriti-final/train.csv'
test_path = '/kaggle/input/kriti-final/test.csv'

train_df =  pd.read_csv(train_path)
test_df=pd.read_csv(test_path)
train_df.shape

# Importing train and test data

(51210, 4)

In [7]:
import ast
train_df['Categories'] = train_df['Categories'].apply(ast.literal_eval)
dummies = pd.get_dummies(train_df['Categories'].explode()).groupby(level=0).sum()
train_df = train_df.join(dummies)
train_df.head()

#  One-hot encoding the categories into 57 columns

,Id,Title,Abstract,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,9707,Axiomatic Aspects of Default Inference,This paper studies axioms for nonmonotonic con...,[cs.LO],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24198,On extensions of group with infinite conjugacy...,We characterize the group property of being wi...,[math.GR],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35766,An Analysis of Complex-Valued CNNs for RF Data...,Recent deep neural network-based device classi...,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14322,On the reconstruction of the drift of a diffus...,The problem of reconstructing the drift of a d...,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,709,Three classes of propagation rules for GRS and...,"In this paper, we study the Hermitian hulls of...","[cs.IT, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
target_list = [label for label in train_df.columns if label not in ['Id','Abstract','Categories','Title']]
target_list

# Making a list of all 57 categories

['cs.AI',
 'cs.AR',
 'cs.CE',
 'cs.CL',
 'cs.CR',
 'cs.CV',
 'cs.DB',
 'cs.DC',
 'cs.DM',
 'cs.GT',
 'cs.IR',
 'cs.IT',
 'cs.LG',
 'cs.LO',
 'cs.NI',
 'cs.OS',
 'cs.PL',
 'cs.RO',
 'cs.SD',
 'cs.SE',
 'econ.EM',
 'econ.GN',
 'econ.TH',
 'eess.AS',
 'eess.IV',
 'eess.SP',
 'math.AC',
 'math.AP',
 'math.AT',
 'math.CO',
 'math.CV',
 'math.GR',
 'math.IT',
 'math.LO',
 'math.NT',
 'math.PR',
 'math.QA',
 'math.ST',
 'q-bio.BM',
 'q-bio.CB',
 'q-bio.GN',
 'q-bio.MN',
 'q-bio.NC',
 'q-bio.TO',
 'q-fin.CP',
 'q-fin.EC',
 'q-fin.GN',
 'q-fin.MF',
 'q-fin.PM',
 'q-fin.PR',
 'q-fin.RM',
 'q-fin.TR',
 'stat.AP',
 'stat.CO',
 'stat.ME',
 'stat.ML',
 'stat.TH']

In [9]:
target_frame = train_df[target_list]

In [10]:
column_sums = target_frame.sum()

In [11]:
train_df

,Id,Title,Abstract,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,9707,Axiomatic Aspects of Default Inference,This paper studies axioms for nonmonotonic con...,[cs.LO],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24198,On extensions of group with infinite conjugacy...,We characterize the group property of being wi...,[math.GR],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,35766,An Analysis of Complex-Valued CNNs for RF Data...,Recent deep neural network-based device classi...,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,14322,On the reconstruction of the drift of a diffus...,The problem of reconstructing the drift of a d...,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,709,Three classes of propagation rules for GRS and...,"In this paper, we study the Hermitian hulls of...","[cs.IT, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51205,46266,Generalized Fourier Integral Operators on spac...,Generalized Fourier integral operators (FIOs) ...,[math.AP],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51206,55910,Weakly-Supervised 3D Visual Grounding based on...,Learning to ground natural language queries to...,"[cs.CV, cs.CL]",0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
51207,10971,Strongly pseudoconvex handlebodies,We give an explicit construction of special st...,[math.CV],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
51208,30982,Improving End-to-End Speech Processing by Effi...,Training a high performance end-to-end speech ...,"[cs.CL, cs.LG, cs.SD, eess.AS]",0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
train_df['CONTEXT'] = train_df['Title'] 
test_df['CONTEXT'] =  test_df[ 'Title']

# Making another column namely CONTEXT which is no other than title as we are going to dop title and abstract afterwards

In [13]:
train_df.head()

,Id,Title,Abstract,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,9707,Axiomatic Aspects of Default Inference,This paper studies axioms for nonmonotonic con...,[cs.LO],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Axiomatic Aspects of Default Inference
1,24198,On extensions of group with infinite conjugacy...,We characterize the group property of being wi...,[math.GR],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,On extensions of group with infinite conjugacy...
2,35766,An Analysis of Complex-Valued CNNs for RF Data...,Recent deep neural network-based device classi...,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,An Analysis of Complex-Valued CNNs for RF Data...
3,14322,On the reconstruction of the drift of a diffus...,The problem of reconstructing the drift of a d...,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,On the reconstruction of the drift of a diffus...
4,709,Three classes of propagation rules for GRS and...,"In this paper, we study the Hermitian hulls of...","[cs.IT, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,Three classes of propagation rules for GRS and...


In [14]:
train_df['CONTEXT'] = train_df['CONTEXT'].str.lower()
test_df['CONTEXT'] = test_df['CONTEXT'].str.lower()

# Converting all uppercase words to lowercase in CONTEXT column

In [15]:
# Define the set of manual stopwords
manual_stopwords = set([
    "i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours",
    "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers",
    "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves",
    "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are",
    "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does",
    "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until",
    "while", "of", "at", "by", "for", "with", "about", "against", "between", "into",
    "through", "during", "before", "after", "above", "below", "to", "from", "up", "down",
    "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here",
    "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more",
    "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so",
    "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"
])

def remove_stopwords(text):
    tokens = text.lower().split()
    filtered_tokens = [word for word in tokens if word not in manual_stopwords]
    return ' '.join(filtered_tokens)
train_df['Cleaned_Context'] = train_df['CONTEXT'].astype(str).apply(remove_stopwords)
train_df[['CONTEXT', 'Cleaned_Context']].head()

# Removing stopwords from CONTEXT column

,CONTEXT,Cleaned_Context
0,axiomatic aspects of default inference,axiomatic aspects default inference
1,on extensions of group with infinite conjugacy...,"extensions group infinite conjugacy classes,"
2,an analysis of complex-valued cnns for rf data...,analysis complex-valued cnns rf data-driven wi...
3,on the reconstruction of the drift of a diffus...,reconstruction drift diffusion transition prob...
4,three classes of propagation rules for grs and...,three classes propagation rules grs egrs codes...


In [16]:
train_df

,Id,Title,Abstract,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,...,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT,Cleaned_Context
0,9707,Axiomatic Aspects of Default Inference,This paper studies axioms for nonmonotonic con...,[cs.LO],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,axiomatic aspects of default inference,axiomatic aspects default inference
1,24198,On extensions of group with infinite conjugacy...,We characterize the group property of being wi...,[math.GR],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,on extensions of group with infinite conjugacy...,"extensions group infinite conjugacy classes,"
2,35766,An Analysis of Complex-Valued CNNs for RF Data...,Recent deep neural network-based device classi...,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,an analysis of complex-valued cnns for rf data...,analysis complex-valued cnns rf data-driven wi...
3,14322,On the reconstruction of the drift of a diffus...,The problem of reconstructing the drift of a d...,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,on the reconstruction of the drift of a diffus...,reconstruction drift diffusion transition prob...
4,709,Three classes of propagation rules for GRS and...,"In this paper, we study the Hermitian hulls of...","[cs.IT, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,three classes of propagation rules for grs and...,three classes propagation rules grs egrs codes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51205,46266,Generalized Fourier Integral Operators on spac...,Generalized Fourier integral operators (FIOs) ...,[math.AP],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,generalized fourier integral operators on spac...,generalized fourier integral operators spaces ...
51206,55910,Weakly-Supervised 3D Visual Grounding based on...,Learning to ground natural language queries to...,"[cs.CV, cs.CL]",0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,weakly-supervised 3d visual grounding based on...,weakly-supervised 3d visual grounding based vi...
51207,10971,Strongly pseudoconvex handlebodies,We give an explicit construction of special st...,[math.CV],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,strongly pseudoconvex handlebodies,strongly pseudoconvex handlebodies
51208,30982,Improving End-to-End Speech Processing by Effi...,Training a high performance end-to-end speech ...,"[cs.CL, cs.LG, cs.SD, eess.AS]",0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,improving end-to-end speech processing by effi...,improving end-to-end speech processing efficie...


In [17]:
test_df['Cleaned_Context'] = test_df['CONTEXT'].astype(str).apply(remove_stopwords)
test_df[['CONTEXT', 'Cleaned_Context']].head()

,CONTEXT,Cleaned_Context
0,pricing fx options under intermediate currency,pricing fx options intermediate currency
1,a multicore processor based real-time system f...,multicore processor based real-time system aut...
2,perceptual quality improvement in videoconfere...,perceptual quality improvement videoconferenci...
3,hundred-kilobyte lookup tables for efficient s...,hundred-kilobyte lookup tables efficient singl...
4,efficient sequence labeling with actor-critic ...,efficient sequence labeling actor-critic training


In [18]:
train_df[['CONTEXT', 'Cleaned_Context']]

,CONTEXT,Cleaned_Context
0,axiomatic aspects of default inference,axiomatic aspects default inference
1,on extensions of group with infinite conjugacy...,"extensions group infinite conjugacy classes,"
2,an analysis of complex-valued cnns for rf data...,analysis complex-valued cnns rf data-driven wi...
3,on the reconstruction of the drift of a diffus...,reconstruction drift diffusion transition prob...
4,three classes of propagation rules for grs and...,three classes propagation rules grs egrs codes...
...,...,...
51205,generalized fourier integral operators on spac...,generalized fourier integral operators spaces ...
51206,weakly-supervised 3d visual grounding based on...,weakly-supervised 3d visual grounding based vi...
51207,strongly pseudoconvex handlebodies,strongly pseudoconvex handlebodies
51208,improving end-to-end speech processing by effi...,improving end-to-end speech processing efficie...


In [19]:
train_df['CONTEXT'] = train_df['Cleaned_Context']
train_df.drop('Cleaned_Context',axis = 1 , inplace = True)
train_df.head()

,Id,Title,Abstract,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,9707,Axiomatic Aspects of Default Inference,This paper studies axioms for nonmonotonic con...,[cs.LO],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,axiomatic aspects default inference
1,24198,On extensions of group with infinite conjugacy...,We characterize the group property of being wi...,[math.GR],0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"extensions group infinite conjugacy classes,"
2,35766,An Analysis of Complex-Valued CNNs for RF Data...,Recent deep neural network-based device classi...,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,analysis complex-valued cnns rf data-driven wi...
3,14322,On the reconstruction of the drift of a diffus...,The problem of reconstructing the drift of a d...,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,reconstruction drift diffusion transition prob...
4,709,Three classes of propagation rules for GRS and...,"In this paper, we study the Hermitian hulls of...","[cs.IT, math.IT]",0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,three classes propagation rules grs egrs codes...


In [20]:
test_df['CONTEXT'] = test_df['Cleaned_Context']
test_df.drop('Cleaned_Context',axis = 1 , inplace = True)
test_df.head()

,Id,Title,Abstract,CONTEXT
0,30332,Pricing FX Options under Intermediate Currency,We suggest an intermediate currency approach t...,pricing fx options intermediate currency
1,50337,A Multicore Processor based Real-Time System f...,In this paper we propose an Intelligent Manage...,multicore processor based real-time system aut...
2,66515,Perceptual Quality Improvement in Videoconfere...,"In the latest years, videoconferencing has tak...",perceptual quality improvement videoconferenci...
3,57464,Hundred-Kilobyte Lookup Tables for Efficient S...,Conventional super-resolution (SR) schemes mak...,hundred-kilobyte lookup tables efficient singl...
4,43169,Efficient Sequence Labeling with Actor-Critic ...,Neural approaches to sequence labeling often u...,efficient sequence labeling actor-critic training


In [21]:
train_df.drop(['Title','Abstract'],axis= 1 , inplace =  True)
train_df.head()

# Dropping Title and Abstract column from train dataset

,Id,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,9707,[cs.LO],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,axiomatic aspects default inference
1,24198,[math.GR],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,"extensions group infinite conjugacy classes,"
2,35766,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,analysis complex-valued cnns rf data-driven wi...
3,14322,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,reconstruction drift diffusion transition prob...
4,709,"[cs.IT, math.IT]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,three classes propagation rules grs egrs codes...


In [22]:
test_df.drop(['Title','Abstract'],axis= 1 , inplace =  True)
test_df.head()

# Dropping Title and Abstract column from test dataset

,Id,CONTEXT
0,30332,pricing fx options intermediate currency
1,50337,multicore processor based real-time system aut...
2,66515,perceptual quality improvement videoconferenci...
3,57464,hundred-kilobyte lookup tables efficient singl...
4,43169,efficient sequence labeling actor-critic training


In [23]:
import re
train_df['CONTEXT'] = train_df['CONTEXT'].str.replace(r'[^\w\s]', '', regex=True)
train_df.head()

# Removing alphanumeric and whitespace characters from train

,Id,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,9707,[cs.LO],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,axiomatic aspects default inference
1,24198,[math.GR],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,extensions group infinite conjugacy classes
2,35766,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,analysis complexvalued cnns rf datadriven wire...
3,14322,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,reconstruction drift diffusion transition prob...
4,709,"[cs.IT, math.IT]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,three classes propagation rules grs egrs codes...


In [24]:
test_df['CONTEXT'] = test_df['CONTEXT'].str.replace(r'[^\w\s]', '', regex=True)
test_df.head()

# Removing alphanumeric and whitespace characters from test

,Id,CONTEXT
0,30332,pricing fx options intermediate currency
1,50337,multicore processor based realtime system auto...
2,66515,perceptual quality improvement videoconferenci...
3,57464,hundredkilobyte lookup tables efficient single...
4,43169,efficient sequence labeling actorcritic training


In [25]:
from sklearn.model_selection import train_test_split

In [26]:
sampled_data = train_df

In [27]:
sampled_data

,Id,Categories,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,9707,[cs.LO],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,axiomatic aspects default inference
1,24198,[math.GR],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,extensions group infinite conjugacy classes
2,35766,"[cs.LG, cs.IT, eess.SP, math.IT]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,analysis complexvalued cnns rf datadriven wire...
3,14322,"[math.PR, math.ST, stat.TH]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,reconstruction drift diffusion transition prob...
4,709,"[cs.IT, math.IT]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,three classes propagation rules grs egrs codes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51205,46266,[math.AP],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,generalized fourier integral operators spaces ...
51206,55910,"[cs.CV, cs.CL]",0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,weaklysupervised 3d visual grounding based vis...
51207,10971,[math.CV],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,strongly pseudoconvex handlebodies
51208,30982,"[cs.CL, cs.LG, cs.SD, eess.AS]",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,improving endtoend speech processing efficient...


In [28]:
sampled_data.drop(labels=['Categories','Id'], axis=1, inplace=True)
sampled_data.head()

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,axiomatic aspects default inference
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,extensions group infinite conjugacy classes
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,analysis complexvalued cnns rf datadriven wire...
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,reconstruction drift diffusion transition prob...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,three classes propagation rules grs egrs codes...


In [29]:
train_percentage = 0.90
val_percentage = 0.05
test_percentage = 0.05
train_data, temp_data = train_test_split(sampled_data, test_size=(1 - train_percentage))
val_data, test_data = train_test_split(temp_data, test_size=(test_percentage / (test_percentage + val_percentage)))

# Splitting train dataset into train, val and test dataset

In [30]:
train_data

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
45491,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,insertonly versus insertdelete dynamic query e...
8064,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,multitask learning minimally invasive surgical...
41031,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,lattice closure relations poset
13126,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pac learning algorithm ltl omegaregular object...
39344,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,operads algebraic homotopy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25258,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,normalised discounted cumulative gain offpolic...
46142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,increasing dimension asymptotics twoway crosse...
25038,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,termination analysis general logic programs mo...
50196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,learning spectrotemporal representations compl...


In [31]:
max_length = train_data['CONTEXT'].apply(len).max()
print("Maximum length of text in the column:", max_length)
average_length = train_data['CONTEXT'].apply(len).mean()
print("Average length of text in the column:", average_length)

# Calculating max length and average length of text in CONTEXT

Maximum length of text in the column: 199
Average length of text in the column: 62.01338714226822


In [32]:
MAX_LEN = 30
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 5
LEARNING_RATE = 5e-05

# Defining hyperparameters for training

In [33]:
from transformers import AutoModel, AutoTokenizer

model_path = "/kaggle/input/bert-base-uncased"
tokenizer_path = "/kaggle/input/bert-base-uncased"

model = AutoModel.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Importing BERT model from local files

In [34]:
from transformers import BertModel

In [35]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['CONTEXT']
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }
    
# Creating a function CustomDataset for tokenizing

In [36]:
train_data = train_data.reset_index(drop = True)
val_data = val_data.reset_index(drop = True)

In [37]:
train_dataset = CustomDataset(train_data, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_data, tokenizer, MAX_LEN)

In [38]:
train_data.head()

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,insertonly versus insertdelete dynamic query e...
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,multitask learning minimally invasive surgical...
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,lattice closure relations poset
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,pac learning algorithm ltl omegaregular object...
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,operads algebraic homotopy


In [39]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False
)

# Creating data loader for train and val

In [40]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)
        
# Creating load and save checkpoint to save progress after every epoch

In [42]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained('/kaggle/input/bert-base-uncased', return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 57)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

# Defining BERT model with every layer

BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [43]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np


class ResampleLoss(nn.Module):
    def __init__(
        self,
        use_sigmoid=True,
        partial=False,
        loss_weight=1.0,
        reduction="mean",
        reweight_func=None,
        weight_norm=None,
        focal=dict(focal=True, alpha=0.5, gamma=2),
        map_param=dict(alpha=10.0, beta=0.2, gamma=0.1),
        CB_loss=dict(CB_beta=0.9, CB_mode="average_w"),
        logit_reg=dict(neg_scale=5.0, init_bias=0.1),
        class_freq=None,
        train_num=None,
    ):
        super(ResampleLoss, self).__init__()
        assert (use_sigmoid is True) or (partial is False)
        self.use_sigmoid = use_sigmoid
        self.partial = partial
        self.loss_weight = loss_weight
        self.reduction = reduction
        if self.use_sigmoid:
            if self.partial:
                self.cls_criterion = partial_cross_entropy
            else:
                self.cls_criterion = binary_cross_entropy
        else:
            self.cls_criterion = cross_entropy
        # reweighting function
        self.reweight_func = reweight_func
        # normalization (optional)
        self.weight_norm = weight_norm
        # focal loss params
        self.focal = focal["focal"]
        self.gamma = focal["gamma"]
        self.alpha = focal["alpha"]
        # mapping function params
        self.map_alpha = map_param["alpha"]
        self.map_beta = map_param["beta"]
        self.map_gamma = map_param["gamma"]
        # CB loss params (optional)
        self.CB_beta = CB_loss["CB_beta"]
        self.CB_mode = CB_loss["CB_mode"]
        self.class_freq = (
            torch.from_numpy(np.asarray(class_freq)).float().cuda()
        )
        self.num_classes = self.class_freq.shape[0]
        self.train_num = train_num  # only used to be divided by class_freq
        # regularization params
        self.logit_reg = logit_reg
        self.neg_scale = (
            logit_reg["neg_scale"] if "neg_scale" in logit_reg else 1.0
        )
        init_bias = (
            logit_reg["init_bias"] if "init_bias" in logit_reg else 0.0
        )
        self.init_bias = (
            -torch.log(self.train_num / self.class_freq - 1) * init_bias
        )
        self.freq_inv = (
            torch.ones(self.class_freq.shape).cuda() / self.class_freq
        )
        self.propotion_inv = self.train_num / self.class_freq

    def forward(
        self,
        cls_score,
        label,
        weight=None,
        avg_factor=None,
        reduction_override=None,
        **kwargs
    ):
        assert reduction_override in (None, "none", "mean", "sum")
        reduction = (
            reduction_override if reduction_override else self.reduction
        )
        weight = self.reweight_functions(label)
        cls_score, weight = self.logit_reg_functions(
            label.float(), cls_score, weight
        )
        if self.focal:
            logpt = self.cls_criterion(
                cls_score.clone(),
                label,
                weight=None,
                reduction="none",
                avg_factor=avg_factor,
            )
            # pt is sigmoid(logit) for pos or sigmoid(-logit) for neg
            pt = torch.exp(-logpt)
            wtloss = self.cls_criterion(
                cls_score, label.float(), weight=weight, reduction="none"
            )
            alpha_t = torch.where(label == 1, self.alpha, 1 - self.alpha)
            loss = alpha_t * ((1 - pt) ** self.gamma) * wtloss
            loss = reduce_loss(loss, reduction)
        else:
            loss = self.cls_criterion(
                cls_score, label.float(), weight, reduction=reduction
            )
        loss = self.loss_weight * loss
        return loss

    def reweight_functions(self, label):
        if self.reweight_func is None:
            return None
        elif self.reweight_func in ["inv", "sqrt_inv"]:
            weight = self.RW_weight(label.float())
        elif self.reweight_func in "rebalance":
            weight = self.rebalance_weight(label.float())
        elif self.reweight_func in "CB":
            weight = self.CB_weight(label.float())
        else:
            return None
        if self.weight_norm is not None:
            if "by_instance" in self.weight_norm:
                max_by_instance, _ = torch.max(weight, dim=-1, keepdim=True)
                weight = weight / max_by_instance
            elif "by_batch" in self.weight_norm:
                weight = weight / torch.max(weight)
        return weight

    def logit_reg_functions(self, labels, logits, weight=None):
        if not self.logit_reg:
            return logits, weight
        if "init_bias" in self.logit_reg:
            logits += self.init_bias
        if "neg_scale" in self.logit_reg:
            logits = logits * (1 - labels) * self.neg_scale + logits * labels
            if weight is not None:
                weight = (
                    weight / self.neg_scale * (1 - labels) + weight * labels
                )
        return logits, weight

    def rebalance_weight(self, gt_labels):
        repeat_rate = torch.sum(
            gt_labels.float() * self.freq_inv, dim=1, keepdim=True
        )
        pos_weight = (
            self.freq_inv.clone().detach().unsqueeze(0) / repeat_rate
        )
        # pos and neg are equally treated
        weight = (
            torch.sigmoid(self.map_beta * (pos_weight - self.map_gamma))
            + self.map_alpha
        )
        return weight

    def CB_weight(self, gt_labels):
        if "by_class" in self.CB_mode:
            weight = (
                torch.tensor((1 - self.CB_beta)).cuda()
                / (1 - torch.pow(self.CB_beta, self.class_freq)).cuda()
            )
        elif "average_n" in self.CB_mode:
            avg_n = torch.sum(
                gt_labels * self.class_freq, dim=1, keepdim=True
            ) / torch.sum(gt_labels, dim=1, keepdim=True)
            weight = (
                torch.tensor((1 - self.CB_beta)).cuda()
                / (1 - torch.pow(self.CB_beta, avg_n)).cuda()
            )
        elif "average_w" in self.CB_mode:
            weight_ = (
                torch.tensor((1 - self.CB_beta)).cuda()
                / (1 - torch.pow(self.CB_beta, self.class_freq)).cuda()
            )
            weight = torch.sum(
                gt_labels * weight_, dim=1, keepdim=True
            ) / torch.sum(gt_labels, dim=1, keepdim=True)
        elif "min_n" in self.CB_mode:
            min_n, _ = torch.min(
                gt_labels * self.class_freq + (1 - gt_labels) * 100000,
                dim=1,
                keepdim=True,
            )
            weight = (
                torch.tensor((1 - self.CB_beta)).cuda()
                / (1 - torch.pow(self.CB_beta, min_n)).cuda()
            )
        else:
            raise NameError
        return weight

    def RW_weight(self, gt_labels, by_class=True):
        if "sqrt" in self.reweight_func:
            weight = torch.sqrt(self.propotion_inv)
        else:
            weight = self.propotion_inv
        if not by_class:
            sum_ = torch.sum(weight * gt_labels, dim=1, keepdim=True)
            weight = sum_ / torch.sum(gt_labels, dim=1, keepdim=True)
        return weight


def reduce_loss(loss, reduction):
    """Reduce loss as specified.
    Args:
        loss (Tensor): Elementwise loss tensor.
        reduction (str): Options are "none", "mean" and "sum".
    Return:
        Tensor: Reduced loss tensor.
    """
    reduction_enum = F._Reduction.get_enum(reduction)
    # none: 0, elementwise_mean:1, sum: 2
    if reduction_enum == 0:
        return loss
    elif reduction_enum == 1:
        return loss.mean()
    elif reduction_enum == 2:
        return loss.sum()


def weight_reduce_loss(loss, weight=None, reduction="mean", avg_factor=None):
    """Apply element-wise weight and reduce loss.
    Args:
        loss (Tensor): Element-wise loss.
        weight (Tensor): Element-wise weights.
        reduction (str): Same as built-in losses of PyTorch.
        avg_factor (float): Avarage factor when computing the mean of losses.
    Returns:
        Tensor: Processed loss values.
    """
    # if weight is specified, apply element-wise weight
    if weight is not None:
        loss = loss * weight
    # if avg_factor is not specified, just reduce the loss
    if avg_factor is None:
        loss = reduce_loss(loss, reduction)
    else:
        # if reduction is mean, then average the loss by avg_factor
        if reduction == "mean":
            loss = loss.sum() / avg_factor
        # if reduction is 'none', then do nothing, otherwise raise an error
        elif reduction != "none":
            raise ValueError(
                'avg_factor can not be used with reduction="sum"'
            )
    return loss


def binary_cross_entropy(
    pred, label, weight=None, reduction="mean", avg_factor=None
):
    # weighted element-wise losses
    if weight is not None:
        weight = weight.float()
    loss = F.binary_cross_entropy_with_logits(
        pred, label.float(), weight, reduction="none"
    )
    loss = weight_reduce_loss(
        loss, reduction=reduction, avg_factor=avg_factor
    )

    return loss
# Defining a custom loss function

In [44]:
column_sums = train_df[target_list].sum()
sums_list = column_sums.tolist()
len(sums_list)

57

In [45]:
resamp_loss = ResampleLoss(
    reweight_func="rebalance",
    loss_weight=1.0,
    focal=dict(focal=True, alpha=0.5, gamma=2),
    logit_reg=dict(init_bias=0.05, neg_scale=2.0),
    map_param=dict(alpha=0.1, beta=10.0, gamma=0.405),
    class_freq=sums_list,
    train_num=len(train_data),
)

# Inserting hyperparameters for loss function

In [46]:
def loss_fn(outputs, targets):
    return resamp_loss(outputs,targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

# Initializing loss function and optimizer for training

In [47]:
val_targets=[]
val_outputs=[]

In [48]:
ckpt_path = "/kaggle/working/curr"
best_model_path = "/kaggle/working/best.pt"

# Intializing checkpoint and model path for saving progress

In [49]:
import torch
torch.cuda.empty_cache()

In [50]:
from tqdm.notebook import tqdm
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0
    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in tqdm(enumerate(training_loader), total=len(training_loader), desc=f"Epoch {epoch} Training"):
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in tqdm(enumerate(validation_loader), total=len(validation_loader), desc=f"Epoch {epoch} Validation"):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

# Defining train function

In [52]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)
# Training the model on the split train and val dataset

############# Epoch 1: Training Start   #############


Epoch 1 Training:   0%|          | 0/1441 [00:00<?, ?it/s]

############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############


Epoch 1 Validation:   0%|          | 0/80 [00:00<?, ?it/s]

############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.000003 	Average Validation Loss: 0.000044
Validation loss decreased (inf --> 0.000044).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############


...

############# Epoch 4: Validation End     #############
Epoch: 4 	Avgerage Training Loss: 0.000002 	Average Validation Loss: 0.000044
############# Epoch 4  Done   #############

############# Epoch 5: Training Start   #############


Epoch 5 Training:   0%|          | 0/1441 [00:00<?, ?it/s]

############# Epoch 5: Training End     #############
############# Epoch 5: Validation Start   #############


Epoch 5 Validation:   0%|          | 0/80 [00:00<?, ?it/s]

############# Epoch 5: Validation End     #############
Epoch: 5 	Avgerage Training Loss: 0.000001 	Average Validation Loss: 0.000048
############# Epoch 5  Done   #############



In [53]:
local_test_data = test_data
# Created local test data from split test dataset

In [54]:
local_test_data.index.name = 'Id'
local_test_data.reset_index(inplace=True)

In [55]:
local_test_data

,Id,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,...,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH,CONTEXT
0,727,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,asr data augmentation lowresource settings usi...
1,39666,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,behaviour description database avs singapore
2,12918,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,understanding robustness difference stochastic...
3,10454,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,attention need blind room volume estimation
4,13961,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,multiview contrastive learning entity typing k...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556,30772,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,asymptotic normality nonparametric kernel type...
2557,14865,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,landesmanlaser conditions quasilinear elliptic...
2558,21999,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,generative speech recognition error correction...
2559,46982,0,0,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,1,0,accurate scalable verifiable protocol federate...


In [56]:
local_test_data_target = local_test_data[target_list]

In [57]:
local_test_data_features = local_test_data[['Id','CONTEXT']]

In [58]:
local_test_data_features

,Id,CONTEXT
0,727,asr data augmentation lowresource settings usi...
1,39666,behaviour description database avs singapore
2,12918,understanding robustness difference stochastic...
3,10454,attention need blind room volume estimation
4,13961,multiview contrastive learning entity typing k...
...,...,...
2556,30772,asymptotic normality nonparametric kernel type...
2557,14865,landesmanlaser conditions quasilinear elliptic...
2558,21999,generative speech recognition error correction...
2559,46982,accurate scalable verifiable protocol federate...


In [59]:
local_test_data_target

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2556,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2558,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2559,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [60]:
class PrepForPredictionDataset(torch.utils.data.Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.title = dataframe['CONTEXT']
        #self.targets = self.data.target_list
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
           # 'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }
# Tokenizing the local test data

In [61]:
local_testing_set = PrepForPredictionDataset(local_test_data, tokenizer, MAX_LEN)
local_testing_set[0]

{'ids': tensor([  101,  2004,  2099,  2951, 15476,  3672,  3370,  2659,  6072,  8162,
          3401, 10906,  2478,  2892,  2989,  8787,  4800, 13102, 25508,  2121,
         23746,  2015,  2892,  2989,  8787,  2376,  7584,   102,     0,     0]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 0, 0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0])}

In [62]:
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

In [63]:
local_testing_loader = DataLoader(local_testing_set, batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0)

# Using the data loader on local test data

In [64]:
def do_prediction(loader):
    model.eval()
   
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(loader):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            #targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            #fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs

# Predicting the output of local test data

In [81]:
local_pred_outputs = do_prediction(local_testing_loader)

In [82]:
type(local_pred_outputs) 

list

In [83]:
len(local_pred_outputs)

2561

In [84]:
local_test_targets_array = np.array(local_test_data_target)

In [85]:
local_test_targets_array[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [86]:
local_pred_outputs_array = np.array(local_pred_outputs)

In [96]:
local_pred_outputs_array

array([[0.41582584, 0.16051538, 0.18001783, ..., 0.18250065, 0.40006113,
        0.18413985],
       [0.4935855 , 0.2417461 , 0.43238673, ..., 0.15513253, 0.24540143,
        0.16670325],
       [0.53516912, 0.20982575, 0.2771481 , ..., 0.33398649, 0.46856341,
        0.3040365 ],
       ...,
       [0.44669363, 0.13705732, 0.15423323, ..., 0.17967117, 0.34340841,
        0.1813418 ],
       [0.36561623, 0.18910737, 0.22350825, ..., 0.27844751, 0.41306752,
        0.21648097],
       [0.37214804, 0.16017696, 0.20215668, ..., 0.17894144, 0.25600344,
        0.19182704]])

In [97]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

In [98]:
local_pred_outputs = (np.array(local_pred_outputs) >= 0.49).astype(int)
# Setting the threshold of probabilities as 0.49

In [99]:
local_pred_outputs

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [100]:
local_preds_df = pd.DataFrame(local_pred_outputs, columns =target_list) 
local_preds_df.head()

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [101]:
local_preds_df.shape

(2561, 57)

In [102]:
import sklearn
from sklearn.metrics import accuracy_score, f1_score

f1_score_micro = f1_score(local_test_targets_array, local_pred_outputs, average='micro')
f1_score_macro = f1_score(local_test_targets_array, local_pred_outputs, average='macro')

print(f"F1 Score (Micro) = {f1_score_micro}")
print(f"F1 Score (Macro) = {f1_score_macro}")

# Calculating the micro and macro F1 score for local test data

F1 Score (Micro) = 0.6356791486798515
F1 Score (Macro) = 0.5625947503643676


In [103]:
test_df['CONTEXT'] = test_df['CONTEXT'].str.lower()

In [105]:
test_df.head()

,Id,CONTEXT
0,30332,pricing fx options intermediate currency
1,50337,multicore processor based realtime system auto...
2,66515,perceptual quality improvement videoconferenci...
3,57464,hundredkilobyte lookup tables efficient single...
4,43169,efficient sequence labeling actorcritic training


In [106]:
actual_test_data = test_df
actual_test_data

,Id,CONTEXT
0,30332,pricing fx options intermediate currency
1,50337,multicore processor based realtime system auto...
2,66515,perceptual quality improvement videoconferenci...
3,57464,hundredkilobyte lookup tables efficient single...
4,43169,efficient sequence labeling actorcritic training
...,...,...
10969,41708,new lipidstructured model investigate opposing...
10970,38843,evaluating efficacy hybrid deep learning model...
10971,57571,weakly supervised video individual countingwea...
10972,31964,structformer learning spatial structure langua...


In [107]:
class PrepForPredictionDataset(torch.utils.data.Dataset):

    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.title = dataframe['CONTEXT']
        #self.targets = self.data.target_list
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
           # 'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }
    
# Tokenizing the test data

In [108]:
actual_testing_set = PrepForPredictionDataset(actual_test_data, tokenizer, MAX_LEN)
actual_testing_set[0]

{'ids': tensor([  101, 20874, 23292,  7047,  7783,  9598,   102,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]),
 'mask': tensor([1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0]),
 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0])}

In [109]:
actual_testing_loader = DataLoader(actual_testing_set, batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0)

# Using the data loader on test data

In [113]:
def do_prediction(loader):
    model.eval()
   
    fin_outputs=[]
    with torch.no_grad():
        for _, data in enumerate(loader):
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            #targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)
            #fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
    return fin_outputs

In [114]:
actual_pred_outputs = do_prediction(actual_testing_loader)

# Predicting the output of test data

In [115]:
actual_pred_outputs = (np.array(actual_pred_outputs) >= 0.49).astype(int)

# Setting the threshold of probabilities as 0.49

In [116]:
actual_pred_outputs

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [117]:
actual_preds_df = pd.DataFrame(actual_pred_outputs, columns =target_list) 
actual_preds_df.head()

,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,cs.GT,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,0,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [119]:
df_concat = pd.concat([test_df, actual_preds_df], axis=1)
df_concat.head()

,Id,CONTEXT,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,30332,pricing fx options intermediate currency,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,50337,multicore processor based realtime system auto...,0,1,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,66515,perceptual quality improvement videoconferenci...,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,57464,hundredkilobyte lookup tables efficient single...,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43169,efficient sequence labeling actorcritic training,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [120]:
b = df_concat.drop('CONTEXT',axis =1 )
b.head()

,Id,cs.AI,cs.AR,cs.CE,cs.CL,cs.CR,cs.CV,cs.DB,cs.DC,cs.DM,...,q-fin.MF,q-fin.PM,q-fin.PR,q-fin.RM,q-fin.TR,stat.AP,stat.CO,stat.ME,stat.ML,stat.TH
0,30332,0,0,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,0,0,0
1,50337,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,66515,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,57464,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43169,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [121]:
sample_sub = pd.read_csv('/kaggle/input/sample-sub/sample_submission.csv')

In [122]:
b = b[sample_sub.columns]

# Making the columns same in order as in sample submission.csv

In [123]:
b.shape

(10974, 58)

In [124]:
b.head()

,Id,math.AT,stat.AP,cs.AR,math.QA,q-bio.MN,eess.AS,eess.IV,stat.ME,econ.GN,...,cs.SE,cs.RO,stat.TH,cs.DB,math.ST,q-bio.GN,q-fin.PM,q-bio.TO,math.GR,cs.IR
0,30332,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,50337,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,66515,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,57464,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,43169,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
b.to_csv('Kapili.csv',index = False)

# Exporting the dataframe to csv file